# Run all을 눌러 시작

In [1]:
# import selenium and so on
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains


# pandas
import pandas as pd
import time

import os.path

In [2]:
# read csv
df = pd.read_csv('bigkinds_date.csv')
df.loc[0, 'isFinished'] = True
df.to_csv('bigkinds_date.csv', index=False)

In [3]:
def set_date(driver, start, end):
  keypads = [Keys.NUMPAD0, Keys.NUMPAD1, Keys.NUMPAD2, Keys.NUMPAD3, Keys.NUMPAD4, Keys.NUMPAD5, Keys.NUMPAD6, Keys.NUMPAD7, Keys.NUMPAD8, Keys.NUMPAD9]

  # repeat 10 times
  for i in range(10):
    driver.find_element_by_id('search-begin-date').send_keys(Keys.BACK_SPACE)

  for i in range(len(start)):
    # character at index i
    pad = int(start[i])
    driver.find_element_by_id('search-begin-date').send_keys(keypads[pad])

  for i in range(10):
    driver.find_element_by_id('search-end-date').send_keys(Keys.BACK_SPACE)

  for i in range(len(end)):
    # character at index i
    pad = int(end[i])
    driver.find_element_by_id('search-end-date').send_keys(keypads[pad])

In [4]:
def login_and_initialize(driver):
  # login
  driver.find_elements_by_class_name('login-modal-btn')[1].click()
  time.sleep(1)
  driver.find_element_by_id('login-user-id').send_keys('hy095875@naver.com')
  driver.find_element_by_id('login-user-password').send_keys('ehddnjs1!')
  driver.find_element_by_id('login-btn').click()
  time.sleep(0.5)

  # click #경제일간지
  driver.find_element_by_id('전국일간지').send_keys(Keys.SPACE)
  driver.find_element_by_id('경제일간지').send_keys(Keys.SPACE)
  driver.find_element_by_id('방송사').send_keys(Keys.SPACE)
  time.sleep(0.5)

  # click '기간' tab
  driver.find_elements_by_class_name('search-tab_group')[0].click()
  time.sleep(0.5)

In [5]:
# get with selenium
chrome_options = webdriver.ChromeOptions()
driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)
driver.get('https://www.bigkinds.or.kr/v2/news/index.do')


# date들을 불러온다
df = pd.read_csv('bigkinds_date.csv')
for i in range(len(df)):
  # 최초 실행시 로그인 및 initialize 수행
  if i == 0:
    login_and_initialize(driver)

  # 이미 수행되었다면 skip(continue)한다
  if df.iloc[i]['isFinished'] == True:
    continue

  # date를 설정한다
  set_date(driver, str(df.iloc[i]['start']), str(df.iloc[i]['end']))

  # 검색 시작
  driver.find_elements_by_class_name('news-report-search-btn')[0].click()


  # 검색 결과 반환되었는지 1초마다 확인한다
  while True:
    time.sleep(1)
    if len(driver.find_elements_by_class_name('news-item')) > 0:
      time.sleep(0.2)
      # 검색 결과가 반환되었으면 아래 코드를 수행(엑셀 다운로드 받기)하고 while문을 빠져나간다
      driver.find_element_by_class_name('step-3-click').click()
      # 정확히는 loading 화면이 종료되어야 한다.
      time.sleep(3)
      # 엑셀 파일을 다운로드 한다.
      driver.find_element_by_css_selector('#analytics-data-download > div.btm-btn-wrp > button').click()
      break


  while True:
    time.sleep(2)
    if os.path.isfile('/Users/tj/Downloads/NewsResult_'+str(df.iloc[i]['start'])+'-'+str(df.iloc[i]['end'])+'.xlsx'):
      break

  # df i번째의 isFinished를 True로 바꾼다
  df.loc[i, 'isFinished'] = True
  # 현 상황을 저장한다
  df.to_csv('bigkinds_date.csv', index=False)

  time.sleep(2)
  # 검색 초기 화면으로 되돌아간다

  element = driver.find_element_by_id("collapse-step-1")
  actions = ActionChains(driver)
  actions.move_to_element(element).perform()
  
  element.click()
  time.sleep(1)



[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - Driver [/Users/tj/.wdm/drivers/chromedriver/mac64/103.0.5060.53/chromedriver] found in cache


KeyboardInterrupt: 

In [ ]:
# read xlsx
# df = pd.read_excel('NewsResult_20220704-20220704.xlsx')
# df

/Users/tj/opt/anaconda3/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,뉴스 식별자,일자,언론사,기고자,제목,통합 분류1,통합 분류2,통합 분류3,사건/사고 분류1,사건/사고 분류2,사건/사고 분류3,인물,위치,기관,키워드,특성추출(가중치순 상위 50개),본문,URL,분석제외 여부
0,2.100601e+06,20220704,한국경제,NaN,[이근미 작가의 BOOK STORY] 끝나지 않은 상처를 치유하려는 소녀들의 용기,문화>방송_연예,문화>전시_공연,문화>출판,범죄>범죄일반>폭행,NaN,NaN,"고현,김벼리,김선영,무진",원주,"아토,여울","이근미,작가,BOOK,STORY,상처,치유,소녀들,용기,폐가,구입,엄마,세간살이,한...","벼리,세나,고현,톱스타,아렌트,김선영,국어샘,아이들,무진,원주민,베스트셀러,홀로코스...",“이 집에 살던 열일곱 살 난 딸이 죽었단다.” \n \n갑자기 차를 세우고 둘러봤...,https://www.hankyung.com/life/article/20220701...,NaN
1,2.100601e+06,20220704,한국경제,NaN,[경제학 원론 산책] 수요가 탄력적이면 가격 내려야 기업 수입 늘어나,경제>유통,경제>서비스_쇼핑,경제>자동차,NaN,NaN,NaN,NaN,계산,NaN,"수요,탄력적,가격,기업,수입,가격,수요량,감소,가격,하락,수요량,증가,현상,수요,법...","수요량,변화율,탄력성,수요곡선,비탄력적,elastic,변화량,소비자,구매량,원인변수...","가격이 오르면 수요량은 감소하고, 가격이 하락하면 수요량이 증가하는 현상을 수요의 ...",https://www.hankyung.com/news/article/20220701...,NaN
2,2.100601e+06,20220704,한국경제,NaN,"[윤명철의 한국, 한국인 이야기] 막대한 피해 입은 조선, 승전 패전 주장 엇갈려 ...",국제>일본,국제>아시아,국제>유럽_EU,NaN,NaN,NaN,"이일,이덕형,선조,광해군,신잡,이항복,윤길,김성일,중종","상주,포르투갈,전선,삼포왜란,임해군,동래성,한국,대마도,부산진,함락,남한강,도요토미...","일본군,비변사,일본,조선,한양","윤명철,한국,한국인,이야기,피해,조선,승전,패전,주장,조선군,백성,22만,사망,농토...","조선,일본,임진왜란,군적,일본군,의주,1만,국제전,중종,도요토미,히데요,규슈,통신사...","임진왜란의 결과로 본 조선의 패배임진왜란이 왜 패전인가는 전쟁 발생과 진행 과정, ...",https://www.hankyung.com/news/article/20220701...,NaN
3,2.100601e+06,20220704,한국경제,NaN,[디지털 이코노미] 덴마크 노조가 로봇 도입을 반기는 이유는,사회>노동_복지,NaN,NaN,사회>사회갈등>시위,NaN,NaN,NaN,"덴마크,독일,미국,일본,지구,중국,홍콩,타이완,싱가포르,한국","삼위일체,아시아,덴마크,정부,금속노조,유럽,싱가포르,노조싱가포르","덴마크,노조,로봇,삼각형,자체,완벽,도형,사각형,정사각형,마름모,삼각형,길이,설정,...","노동자,재교육,싱가포르,덴마크,제조업,삼위일체,노동시장,신기술,gdp,노동자들,아시...","삼각형은 그 자체로 완벽한 도형이다. 사각형은 정사각형도 마름모도 될 수 있지만, ...",https://www.hankyung.com/news/article/20220701...,NaN
4,2.100601e+06,20220704,한국경제,NaN,[과학과 놀자] 세포에서 단백질의 발현 이동 분해 보여주는 표지자 역할,IT_과학>과학,사회>사회일반,문화>출판,NaN,NaN,NaN,"오사무,안톤 판 레이우엔훅","지평,미국,지구,호르,네덜란드","보배,한국한의학연구원,일본계,이온","세포,단백질,발현,이동,분해,표지자,역할,백문,불여일견,이야기,일상생활,사용,확인,...","단백질,생물학,과학자,생명과학,몸속,현미경,생명체,표지자,생물학적,gfp,시모무라,...","백문이 불여일견, 보아야 믿지, 보고 나서 이야기하자. 우리가 일상생활에서 빈번하게...",https://www.hankyung.com/news/article/20220701...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8055,1.100611e+06,20220704,서울신문,류지영,‘中민심 지표’ 돼지고기값 일주일 새 13% 급등,국제>중국,경제>유통,경제>금융_재테크,NaN,NaN,NaN,NaN,"우크라,중국,우크라이나 전쟁,한국,중국농업농촌부","펑파이,공산당","민심,지표,돼지고기,일주일,13%,급등,우크라,전쟁,여파,사료,가격,세계,식품,물가...","중국,돼지고기,일주일,코로나19,우크라,펑파이,이듬해,소비국",다른 나라에 비해 소비자 물가가 안정적인 중국에서도 ‘민심의 바로미터’로 불리는 돼...,http://www.seoul.co.kr/news/newsView.php?id=20...,NaN
8056,2.100701e+06,20220704,헤럴드경제,박혜림 기자,"“중국 싫어, 삼성 것 달라” 한국산 판매 중단 난리난 ‘러시아’",IT_과학>모바일,국제>러시아,경제>유통,NaN,NaN,NaN,NaN,"카자흐스탄,우크라이나,중국,한국,러시아","정부,애플,엠비디오,삼성전자,삼성,러시아","중국,삼성,러시아,한국,판매,중단,난리난,삼성폰,구입,난리,러시아,우크라이나,전쟁,...","러시아,스마트폰,삼성,삼성전자,만큼,판매량,병행수입,중국,점유율,수입폰,러시아인들,...",[헤럴드경제=박혜림 기자] “AS 없어도 좋으니 삼성폰만 구입하게 해 달라 난리 난...,http://biz.heraldcorp.com/view.php?ud=20220703...,NaN
8057,2.100701e+06,20220704,헤럴드경제,김민지 기자,“배달음식 돈 아까워 안 먹어요” 심각하네 ‘우르르’ 배달앱 떠난다,경제>서비스_쇼핑,경제>유통,경제>증권_증시,NaN,NaN,NaN,박재진,서울,"MAU,WAU,쿠팡이츠","배달음식,배달앱,서울,거주,직장인,박재진,가명,이달,자신,등급,배달,회원,코로나,시...","배달앱,이용자,쿠팡이츠,코로나,1만,배달비,배달기사,엔데믹,빅데이터,mau,오프라인...",[헤럴드경제=김민지 기자] # 서울에 거주하는 직장인 박재진(가명) 씨는 이달 자신...,http://biz.heraldcorp.com/view.php?ud=20220703...,NaN
8058,2.100701e+06,20220704,헤럴드경제,양영경 기자,뜨겁던 서울 맞나요? 상반기 4곳 빼고 아파트값 다 떨어졌다 [부동산360],경제>부동산,경제>금융_재테크,경제>외환,NaN,NaN,NaN,여경희,"서울,용산구,강남구,은평구,강남권,노원구,경기,서대문구,서초구,동작구,수도권,성북구...","한국부동산원,대한건설정책연구원,한국건설산업연구원,노원구,강북구,부동산R114","서울,상반기,아파트값,부동산360,상반기,서울,자치구,4곳,제외,아파트값,양도소득세...","서울,상반기,부동산,매수세,가능성,거래량,자치구,금리인상,연구원,용산구,노원구,여경...",[헤럴드경제=양영경 기자] 올해 상반기 서울 25개 자치구 중 4곳을 제외한 모든 ...,http://biz.heraldcorp.com/view.php?ud=20220703...,NaN
